svmによる実装

In [1]:
import pandas as pd

test = pd.read_csv('test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [2]:
test = test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,male,34.5,0,0,7.8292
1,893,3,female,47.0,1,0,7.0000
2,894,2,male,62.0,0,0,9.6875
3,895,3,male,27.0,0,0,8.6625
4,896,3,female,22.0,1,1,12.2875


In [3]:
test["Sex"] = test["Sex"].replace(["female", "male"], [0, 1])
test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
dtype: int64

In [4]:
test["Age"] = test["Age"].fillna(test["Age"].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [5]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,1,34.5,0,0,7.8292
1,893,3,0,47.0,1,0,7.0000
2,894,2,1,62.0,0,0,9.6875
3,895,3,1,27.0,0,0,8.6625
4,896,3,0,22.0,1,1,12.2875


In [12]:
train = pd.read_csv('train.csv')
train = train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
train['Sex'] = train['Sex'].replace(['female', 'male'], [0, 1])
train["Age"] = train["Age"].fillna(train["Age"].mean())
train['Fare'] = train['Fare'].fillna(train['Fare'].mean())
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [13]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,22.0,1,0,7.2500
1,2,1,1,0,38.0,1,0,71.2833
2,3,1,3,0,26.0,0,0,7.9250
3,4,1,1,0,35.0,1,0,53.1000
4,5,0,3,1,35.0,0,0,8.0500


In [31]:
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# StandardScalerのインスタンスを作成する
scaler = StandardScaler()

# 学習用のデータを用意する
X = train[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']] # 特徴量
y = train['Survived'] # ラベル

# fitメソッドで学習用のデータの平均と分散を計算する
scaler.fit(X)

# transformメソッドでデータを標準化する
X_scaled = scaler.transform(X)

# SVCのインスタンスを作成する
clf = svm.SVC(gamma="scale")

# fitメソッドで学習させる
clf.fit(X_scaled, y)

# predictメソッドで予測する
test_scaled = scaler.transform(test) # テスト用のデータも標準化する
pred = clf.predict(test_scaled) # 予測する
df = pd.DataFrame(data=pred, columns=['Survived'])
rst = pd.merge(test, df, left_index=True, right_index=True)
rst.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,892,3,1,34.5,0,0,7.8292,0
1,893,3,0,47.0,1,0,7.0000,1
2,894,2,1,62.0,0,0,9.6875,0
3,895,3,1,27.0,0,0,8.6625,0
4,896,3,0,22.0,1,1,12.2875,1


In [32]:
pred = rst[['PassengerId', 'Survived']]
pred.to_csv('pred3.csv', index=False)
pred.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [33]:
rst.groupby('Pclass')['Survived'].mean()

Pclass
1    0.476636
2    0.344086
3    0.316514
Name: Survived, dtype: float64

In [35]:
rst.groupby('Sex')['Survived'].mean()

Sex
0    0.927632
1    0.041353
Name: Survived, dtype: float64